# USG grants crawl
## Word Tokens

### Previously

In the previous chapter we loaded the [grants.gov](https://www.grants.gov/web/grants) grants database and looked at some summary characteristics of what it conatined.

In this chapter we'll move on to looking at the available data with the goal of learning more about how government grant solicitations might be targeting open Science

### Loading the database once more

Let's begin by loading up the database provided by the website, which is stored in an xml format.

In [4]:
from bs4 import BeautifulSoup
import xmltodict
import sys

# FUTURE NOTE: it may be possible to do a check for a local file meeting the relevant criterion and conditionally 
# download from https://www.grants.gov/extract/ (and extract compressed file) in the event a local target isn't found.
# For the moment though...

# load up the xml file; hard-path to local file.  Adjust as necessary
pathToXML='C://Users//dbullock//Documents//code//gitDir//USG_grants_crawl//inputData//GrantsDBExtract20230113v2.xml'

# open and parse file
with open(pathToXML, 'r') as f:
    govGrantData_raw = f.read()

# convert xml to dictionary
with open(pathToXML) as xml_file:
    govGrantData_dictionary = xmltodict.parse(xml_file.read())

# quick size legibility function generated by code-davinci-002
def convert_bytes(bytes):
    if bytes < 1024:
        return str(bytes) + " B"
    elif bytes < 1048576:
        return str(round(bytes/1024, 1)) + " KB"
    elif bytes < 1073741824:
        return str(round(bytes/1048576, 1)) + " MB"
    elif bytes < 1099511627776:
        return str(round(bytes/1073741824, 1)) + " GB"
    else:
        return str(round(bytes/1099511627776, 1)) + " TB"
    
# terminal reports
print('Dictionary conversion successful')
print('\n' + str(len(govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'])) + ' grant entries found, totalling '+ convert_bytes(sys.getsizeof(govGrantData_raw)))
#print('\n and with dictionary keys:\n')
#print(govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'][0].keys())

Dictionary conversion successful

70330 grant entries found, totalling 256.2 MB

 and with dictionary keys:

dict_keys(['OpportunityID', 'OpportunityTitle', 'OpportunityNumber', 'OpportunityCategory', 'FundingInstrumentType', 'CategoryOfFundingActivity', 'CategoryExplanation', 'CFDANumbers', 'EligibleApplicants', 'AdditionalInformationOnEligibility', 'AgencyCode', 'AgencyName', 'PostDate', 'CloseDate', 'LastUpdatedDate', 'AwardCeiling', 'AwardFloor', 'EstimatedTotalProgramFunding', 'ExpectedNumberOfAwards', 'Description', 'Version', 'CostSharingOrMatchingRequirement', 'ArchiveDate', 'GrantorContactEmail', 'GrantorContactEmailDescription', 'GrantorContactText'])


In [184]:
import re
# ok we can also try and use these co occurances to threshold the grants and find other shared words

grantIDholder=[]
# begin by using the same code as above to identify grants which occur more than a certian number of times
for iIndexX, iKeywordsX in enumerate(list(grantFindsOut.keys())):
        # get the values for each "node" (e.g. the grant IDs for each keyword)
        IDsX=grantFindsOut[iKeywordsX]
        grantIDholder.extend(IDsX)

# now that we have a vector with those ID's let's get the description content

#actually, we first have to get a list of the grant IDs
allGrantIDs=[]
for iListing in govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0']:
    allGrantIDs.append(iListing['OpportunityID'])

#create a holder
grantDescriptions=[]

for iGrantIDs in grantIDholder:
    currentGrantIndex=allGrantIDs.index(iGrantIDs)
    currentDescription=govGrantData_dictionary['Grants']['OpportunitySynopsisDetail_1_0'][currentGrantIndex]['Description']
    grantDescriptions.append(currentDescription.lower())
    
omnibusWordBag=' '.join(grantDescriptions)

tokenizedWords = re.sub(r"[^a-zA-Z0-9]", " ", omnibusWordBag.lower()).split()

# load the stopword bag
with open('stopwords.txt') as f:
    stopwords = f.read()

# iterate through the stopwords bag
for iStopwords in stopwords.split('\n'):
    iStopwords
    omnibusWordBag=omnibusWordBag.replace(' '+iStopwords+' ',' ')
    omnibusWordBag=omnibusWordBag.replace(' '+iStopwords+'.','')
    
# split the bag of words into individual words
allWordsList=omnibusWordBag.split(' ')

#do the counting
unique_elements, counts_elements = np.unique(allWordsList, return_counts=True)
wordCountsDF=pd.DataFrame(np.asarray([unique_elements,counts_elements]).T,columns=['words','counts'])
#show it
itables.show(wordCountsDF)

MemoryError: Unable to allocate 25.9 GiB for an array with shape (3006099,) and data type <U2311

In [96]:
np.asarray([len(grantFindsOut[iKeyword]) for iKeyword in grantFindsOut.keys()]).T

array([ 119,  154,  524,    1,   37,   66,   60,   23, 1471,    7,  163,
          0,  118,    0,  267,  392,   20,  226,   67,   63, 3626,    9,
        474,  856, 3237,  400,   66,    0])

In [185]:
tokenizedWords

['the',
 'national',
 'historical',
 'publications',
 'and',
 'records',
 'commission',
 'seeks',
 'proposals',
 'that',
 'use',
 'cost',
 'effective',
 'methods',
 'to',
 'digitize',
 'nationally',
 'significant',
 'historical',
 'record',
 'collections',
 'and',
 'make',
 'the',
 'digital',
 'versions',
 'freely',
 'available',
 'online',
 'projects',
 'must',
 'make',
 'use',
 'of',
 'existing',
 'holdings',
 'of',
 'historical',
 'repositories',
 'and',
 'consist',
 'of',
 'entire',
 'collections',
 'or',
 'series',
 'the',
 'materials',
 'should',
 'already',
 'be',
 'available',
 'to',
 'the',
 'public',
 'at',
 'the',
 'archives',
 'and',
 'described',
 'so',
 'that',
 'projects',
 'can',
 're',
 'use',
 'existing',
 'information',
 'to',
 'serve',
 'as',
 'metadata',
 'for',
 'the',
 'digitized',
 'collection',
 'to',
 'make',
 'these',
 'projects',
 'as',
 'widely',
 'useful',
 'as',
 'possible',
 'for',
 'archives',
 'historical',
 'repositories',
 'and',
 'researchers',
 'th

In [186]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print(vectorizer.get_feature_names())

C:\Users\dbullock\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


NotFittedError: Vocabulary not fitted or provided

In [192]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([omnibusWordBag])
print(vectorizer.get_feature_names_out())
X

['00' '000' '000001' ... 'zurab' 'zw' 'zydeco']


<1x41687 sparse matrix of type '<class 'numpy.float64'>'
	with 41687 stored elements in Compressed Sparse Row format>